In [ ]:
! pip install --upgrade --quiet atlas_schema
! pip install --upgrade --quiet --pre mplhep

In [ ]:
import gzip
import json
import re
import time

import awkward as ak
import cloudpickle
import dask
import hist
import matplotlib.pyplot as plt
import mplhep
import numpy as np
import uproot
import vector

from atlas_schema.schema import NtupleSchema
from coffea import processor
from coffea.nanoevents import NanoEventsFactory
from dask.distributed import Client, PipInstall, performance_report


import utils

vector.register_awkward()
mplhep.style.use(mplhep.style.ATLAS1)

client = Client("tls://localhost:8786")

plugin = PipInstall(packages=["atlas_schema"], pip_options=["--upgrade"])
client.register_plugin(plugin)

cloudpickle.register_pickle_by_value(utils)

### fileset preparation

In [ ]:
# load metadata from file
fname = "ntuple_production/file_metadata.json.gz"
with gzip.open(fname) as f:
    dataset_info = json.loads(f.read().decode())

# construct fileset
fileset = {}
input_size_GB = 0
for containers_for_category in dataset_info.values():
    for container, metadata in containers_for_category.items():
        if metadata["files_output"] is None:
            # print(f"skipping missing {container}")
            continue

        dsid, _, campaign = utils.dsid_rtag_campaign(container)

        # debugging shortcuts, use one or both of the following to reduce workload
        if campaign not in ["mc23a", "data22"]: continue
        # if "601229" not in dsid: continue

        weight_xs = utils.sample_xs(campaign, dsid)
        lumi = utils.integrated_luminosity(campaign)
        fileset[container] = {"files": dict((path, "reco") for path in metadata["files_output"]), "metadata": {"dsid": dsid, "campaign": campaign, "weight_xs": weight_xs, "lumi": lumi}}
        input_size_GB += metadata["size_output_GB"]

print(f"fileset has {len(fileset)} categories with {sum([len(f["files"]) for f in fileset.values()])} files total, size is {input_size_GB:.2f} GB")

# minimal fileset for debugging
# fileset = {"mc20_13TeV.601352.PhPy8EG_tW_dyn_DR_incl_antitop.deriv.DAOD_PHYSLITE.e8547_s4231_r13144_p6697": fileset["mc20_13TeV.601352.PhPy8EG_tW_dyn_DR_incl_antitop.deriv.DAOD_PHYSLITE.e8547_s4231_r13144_p6697"]}
# fileset

### simple non-distributed reading

In [ ]:
events = NanoEventsFactory.from_root(
    {list(fileset[list(fileset.keys())[0]]["files"])[0]: "reco"},
    mode="virtual",
    schemaclass=NtupleSchema,
    entry_stop=1000
).events()

h = hist.new.Regular(30, 0, 300, label="leading electron $p_T$").StrCat([], name="variation", growth=True).Weight()

for variation in events.systematic_names:
    if variation != "NOSYS" and "EG_SCALE_ALL" not in variation:
        continue

    cut = events[variation]["pass"]["ejets"] == 1
    h.fill(events[variation][cut==1].el.pt[:, 0] / 1_000, variation=variation)

fig, ax = plt.subplots()
for variation in h.axes[1]:
    h[:, variation].plot(histtype="step", label=variation, ax=ax)
_ = ax.legend()

### pre-processing

In [ ]:
run = processor.Runner(
    executor = processor.DaskExecutor(client=client, treereduction=4),
    # executor = processor.IterativeExecutor(),  # to run locally
    schema=NtupleSchema,
    savemetrics=True,
    chunksize=50_000,
    skipbadfiles=True,
    align_clusters=False,
    # maxchunks=1  # for debugging only
)

with performance_report(filename="preprocess.html"):
    preprocess_output = run.preprocess(fileset)

# write to disk
with open("preprocess_output.json", "w") as f:
    json.dump(utils.preprocess_to_json(preprocess_output), f)

# load from disk
with open("preprocess_output.json") as f:
    preprocess_output = utils.json_to_preprocess(json.load(f))

len(preprocess_output), preprocess_output[:3]

### processing

In [ ]:
class Analysis(processor.ProcessorABC):
    def __init__(self):
        self.h = hist.new.Regular(20, 0, 1_000, label="$m_{jj}$ [GeV]").\
            StrCat([], name="dsid_and_campaign", growth=True).\
            StrCat([], name="variation", growth=True).\
            Weight()

    def process(self, events):
        f = uproot.open(events.metadata["filename"])

        # this should match existing pre-determined metadata
        # sim_type, mc_campaign, dsid, etag = f["metadata"].axes[0]
        # assert mc_campaign == events.metadata["campaign"]
        # assert dsid == events.metadata["dsid"]

        # ensure systematics in schema and in histogram match
        # systematics_from_hist = list(f["listOfSystematics"].axes[0])
        # assert sorted(systematics_from_hist) == sorted(events.systematic_names)

        # categorize events by DSID and campaign with a single histogram axis
        dsid_and_campaign = f"{events.metadata["dsid"]}_{events.metadata["campaign"]}"

        if events.metadata["dsid"] != "data":
            sumw = float(f[f.keys(filter_name="CutBookkeeper*NOSYS")[0]].values()[1])  # initial sum of weights
        else:
            sumw = None  # no normalization for data

        for variation in events.systematic_names:
            if variation not in ["NOSYS"] + [name for name in events.systematic_names if "JET_JER_Effective" in name]:
                continue

            cut = events[variation]["pass"]["ejets"] == 1
            # TODO: remaining weights
            weight = (events[variation][cut==1].weight.mc if events.metadata["dsid"] != "data" else 1.0) * events.metadata["weight_xs"] * events.metadata["lumi"]
            mjj = (events[variation][cut==1].jet[:, 0] + events[variation][cut==1].jet[:, 1]).mass
            self.h.fill(mjj / 1_000, dsid_and_campaign=dsid_and_campaign, variation=variation, weight=weight)

        return {
            "hist": self.h,
            "meta": {
                "sumw": {(events.metadata["dsid"], events.metadata["campaign"]): {(events.metadata["fileuuid"], sumw)}}}  # sumw in a set to avoid summing multiple times per file
        }

    def postprocess(self, accumulator):
        # normalize histograms
        # https://topcptoolkit.docs.cern.ch/latest/starting/running_local/#sum-of-weights
        for dsid_and_campaign in accumulator["hist"].axes[1]:
            dsid, campaign = dsid_and_campaign.split("_")
            if dsid == "data":
                continue  # no normalization for data by total number of weighted events
            norm = 1 / sum([sumw for uuid, sumw in accumulator["meta"]["sumw"][(dsid, campaign)]])
            count_normalized = accumulator["hist"][:, dsid_and_campaign, :].values()*norm
            variance_normalized = accumulator["hist"][:, dsid_and_campaign, :].variances()*norm**2
            accumulator["hist"][:, dsid_and_campaign, :] = np.stack([count_normalized, variance_normalized], axis=-1)


client.run_on_scheduler(utils.start_tracking)  # track worker count on scheduler
t0 = time.perf_counter()  # track walltime

with performance_report(filename="process.html"):
    out, report = run(preprocess_output, processor_instance=Analysis())

t1 = time.perf_counter()
worker_count_dict = client.run_on_scheduler(utils.stop_tracking)  # stop tracking, read out data, get average
nworker_avg = utils.get_avg_num_workers(worker_count_dict)

print(f"histogram size: {out["hist"].view(True).nbytes / 1_000 / 1_000:.2f} GB\n")

# shortened version of report, dropping extra columns
dict((k, v) for k, v in report.items() if k != "columns") | ({"columns": report["columns"][0:10] + ["..."]})

track XCache egress: [link](https://grafana.mwt2.org/d/EKefjM-Sz/af-network-200gbps-challenge?var-cnode=c111_af_uchicago_edu&var-cnode=c112_af_uchicago_edu&var-cnode=c113_af_uchicago_edu&var-cnode=c114_af_uchicago_edu&var-cnode=c115_af_uchicago_edu&viewPanel=195&kiosk=true&orgId=1&from=now-1h&to=now&timezone=browser&refresh=5s)

**to-do for metrics:**
- data rate by tracking `bytesread` per chunk

In [ ]:
print(f"walltime: {t1 - t0:.2f} sec ({(t1 - t0) / 60:.2f} min)")
print(f"average worker count: {nworker_avg:.1f}")
print(f"number of events processed: {report["entries"]:,}\n")

print(f"data read: {report["bytesread"] / 1000**3:.2f} GB in {report["chunks"]} chunks (average {report["bytesread"] / 1000**3 / report["chunks"]:.2f} GB per chunk)")
print(f"average total data rate: {report["bytesread"] / 1000**3 * 8 / (t1 - t0):.2f} Gbps")
print(f"fraction of input files read: {report["bytesread"] / 1000**3 / input_size_GB:.1%}")
print(f"number of branches read: {len(report["columns"])}\n")

print(f"worker-average event rate using \'processtime\': {report["entries"] / 1000 / report["processtime"]:.2f} kHz")
print(f"worker-average data rate using \'processtime\': {report["bytesread"] / 1000**3 * 8 / report["processtime"]:.2f} Gbps\n")

print(f"average event rate using walltime and time-averaged worker count: {report["entries"] / 1000 / (t1 - t0) / nworker_avg:.2f} kHz")
print(f"average data rate using walltime and time-averaged worker count: {report["bytesread"] / 1000**3 * 8 / (t1 - t0) / nworker_avg:.2f} Gbps\n")

print(f"fraction of time spent in processing: {report["processtime"] / ((t1 - t0) * nworker_avg):.1%}")
print(f"average process task length: {report["processtime"] / report["chunks"]:.1f} sec")

_ = utils.plot_worker_count(worker_count_dict)

In [ ]:
mc_stack = []
labels = []
for key in dataset_info:
    dsids = []
    for container in dataset_info[key]:
        dsids.append(container.split(".")[1])

    dsids = sorted(set(dsids))
    dsids_in_hist = [dc for dc in out["hist"].axes[1] if dc.split("_")[0] in dsids]
    # print(f"{key}:\n  - expect {dsids}\n  - have {dsids_in_hist}")

    if key in ["data", "ttbar_H7", "ttbar_hdamp", "ttbar_pthard", "Wt_DS", "Wt_H7", "Wt_pthard"] or len(dsids_in_hist) == 0:
        continue  # data drawn separately, skip MC modeling variations and skip empty categories

    mc_stack.append(out["hist"][:, :, "NOSYS"].integrate("dsid_and_campaign", dsids_in_hist))
    labels.append(key)

try:
    data_hist = out["hist"].integrate("dsid_and_campaign", [dc for dc in out["hist"].axes[1] if "data" in dc])[:, "NOSYS"]
except ValueError:
    print("falling back to plotting first entry of categorical axes as \"data\"")
    data_hist = out["hist"][:, 0, 0]

fig, ax1, ax2 = mplhep.comp.data_model(
    data_hist=data_hist,
    stacked_components=mc_stack,
    stacked_labels=labels,
    # https://scikit-hep.org/mplhep/gallery/model_with_stacked_and_unstacked_histograms_components/
    # unstacked_components=[],
    # unstacked_labels=[],
    xlabel=out["hist"].axes[0].label,
    ylabel="Entries",
)

mplhep.atlas.label("Internal", ax=ax1, data=True, lumi=f"{utils.integrated_luminosity("", total=True) / 1000:.0f}", com="13/ \\ 13.6 \\ TeV")
mplhep.mpl_magic(ax=ax1)
ax2.set_ylim([0.5, 1.5])

# compare to e.g. https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PAPERS/HDBS-2020-11/fig_02a.png
fig.savefig("mjj.png")

In [ ]:
# save to disk
import uhi.io.json

with gzip.open("hist.json.gz", "w") as f:
    f.write(json.dumps(out["hist"], default=uhi.io.json.default).encode("utf-8"))

with gzip.open("hist.json.gz") as f:
    h = hist.Hist(json.loads(f.read(), object_hook=uhi.io.json.object_hook))